# Extract data for desired metrics for 0.25x0.25 degree for vegetation
# Dataset
The MODIS vegetation index (VI) products will provide consistent, spatial and
temporal comparisons of global vegetation conditions which will be used to monitor the Earth's terrestrial photosynthetic vegetation activity in support of phenologic, change detection, and biophysical interpretations. Gridded vegetation index maps depicting spatial and temporal variations in vegetation activity are derived at 16-day and monthly intervals for precise seasonal and interannual monitoring of the Earth’s vegetation.<br><br>
The downloaded data is at 16-day gap and at 0.05x0.05 degree. This has almost 13 scientific datasets out which we are using only NDVI and EVI with valid range as -2000, 10000 and fill value as -3000<br><br>



In [ ]:
import pandas as pd
import numpy as np
import math
import pandas as pd
from pyhdf.SD import SD, SDC
import os
import math

In [ ]:
GRANULES_PATH="" #path where hdf files of MOD13C1v061 is stored.
OUTPUT_PATH="data/" ##where you want to store the output file
LOCATION_FILE = '' #path of files with details of cities with latitude, longitude, population, elevation etc (geonames-all-cities-with-a-population-1000-3.csv).
WEEK_MAPPINGS = 'data/week_mappings_vegetation.csv' ##the week mappings file made from GenerateWeekMappings_vegetation.ipynb. This tells us about which granule falls under which week.
week_data = pd.read_csv(WEEK_MAPPINGS)
week_data=week_data.set_index("week")["granule"].to_dict()

In [ ]:
metrics=["CMG 0.05 Deg 16 days NDVI","CMG 0.05 Deg 16 days EVI"]

# Providing grid names to granules in terms of city from LOCATION_FILE.
Horizontal resolution= 1°x1° degree (360x180)<br>
Upper Left Point= -180.0, 90.0<br>
Lower Right Point= 180.0, -90.0<br>
The latitudes and longitudes of the grid box centers are provided in the data (LATITUDE, LONGITUDE). The upper left box center location is (89.5, -179.5) and the lower right box center location is (-89.5, +179.5). The spatial extent of the 1x1 degree grid spans the upper left (+90, -180) to lower right (-90, +180). The longitude value goes from -180 to 180 and latitude value goes from 90 to -90.<br><br>

The grid's center is given as (latitude, longitude) in data. To map that to out city (latitude, longitude) from LOCATION_FILE do:<br>

1. change latitude and longitude into float<br>
2. Subrtact it from 90 for latitude and from 180 for longitude divide by 0.05 and change it to integer.<br>
3. Divide by main resolution 0.05,multiply by main resolution 0.05
4. Subtract from max value and add the half of the grid 0.025 and round upto 3 places<br>


Say, we have (89.86) latitude. According to (lat, long) center values provided in data, this latitude value is in the grid [89.9, 89.85]. So the center should come as (lat) -> (0.89.875). So the intuition behind using this is, first we have to find how much away we have travelled from max value (here, how much 0.111 is away from 90 by (90 - 89.86 -> 0.14)). How many grids are we away in terms of 0.05 i.e 0.14 / 0.05, change it to integer -> 2. Find how much away we are from 90 in terms of 0.05 by multiplying it with 0.5 -> 0.1, i.e. we are at the center of the previous grid and by add 0.025 (center of 0.05 grid) and subtracting it from 90 we will get center of the grid where our latitude belongs i.e. 89.875<br><br>

Now here the only thing to look when 0.25 degree is to taken into consideration is that total number of 5 grids is needed to make 0.25 degree,length of the block will be 0.25 and center of grid/block will be 0.125

In [ ]:
fp=open(LOCATION_FILE,"r")
evrythng=fp.readlines()
fp.close()
locations=set()
for line in evrythng[1:]:
    line=line.split("\n")[0].split(";")
    [lat,long]=line[-1].split(",")
    lat=round(90-(int((90-float(lat))/0.25)*0.25+0.125),3)
    long=round(180-(int((180-float(long))/0.25)*0.25+0.125),3)
    locations.add((lat,long))

# HDF File
HDF file contains sub datasets of every metrics. To access the metrics of particular latitude and longitude, those latitudes and logitudes are also stored as datasets.<br><br>

We have coordinates of 0.05 degree x 0.05 degree cities. Now we want metrics for these locations (for these latitudes and longitudes).<br>

How to get (latitude, longitude) pair from data for out (latitude, longitude) pair (of cities data)?. The latitude is stored as 2D matrix and also the longitude. We will get lat_obj by selecting latitude and long_obj by selecting longitude.

Let say we want NDVI metric at latitude x and longitude y (i.e. NDVI[x,y]).Run over all indices of latitude and longitude, extract latitude value from latitude matrix for latitude and longitude matrix. Same way extract longitude value from longitude matrix. Compare values with x, y. If matched then store them in location_indices dictionary with key as (x,y) and value as (latitude index, longitude index). Now any metric can be extracted with these location indices.


Total latitude blocks (latitude indices) will be 3600 (90 to -90 with 0.05 as gride) and total longitude blocks (longitude indices) will be 7200 (-180 to 180 as 0.05 gride). The matrices for both latitude and longitude is 3600x7200. Run for loop over latitude range and longitude range, will go through all indices and get latitude and longitude, compare them with our reqiured latitude and longitude values. If matched store them in dictionary.<br><br>

Say to change every index 1 in latitude(3600) as per 0.05 degree, multiply 1 with 0.05 -> 0.05, Subtract it from 90 (max value) and from 0.025 (center of gride) -> 89.925<br><br>

Now here the only thing to look when 0.25 degree is to taken into consideration is that total number of 5 grids is needed to make 0.25 degree,length of the block will be 0.25 and center of grid/block will be 0.125

# Preparing CSV file
Csv file will have the following fields: location, week, metric, mean, sdev, min, max. This data will be made for 0.05 degree x 0.05 degree city.<br>

At the end all the values read above are stored under respective columns in a csv file.<br><br>


In [ ]:
def get_indices(location):
    (lat,long)=location
    lat_start_index=round(((90-lat-0.125)/0.25)*5)
    lat_end_index=lat_start_index+5-1
    long_start_index=round(((long-0.125+180)/0.25)*5)
    long_end_index=long_start_index+5-1
    lat_range=(lat_start_index,lat_end_index)
    long_range=(long_start_index,long_end_index)
    return [lat_range,long_range]

In [ ]:
OUTPUT_FILENAME=OUTPUT_PATH+"indian_cities_0.25degx0.25deg_vegetation.csv"
fp=open(OUTPUT_FILENAME,"w")
fp.write(",".join(["location","week","metric","mean","sdev"])+"\n")
fp.close()
for week in week_data:
    print(week)
    data=[]
    granule=week_data[week]
    hdf = SD(GRANULES_PATH+granule, SDC.READ)
    
    for metric in metrics:
        metric_obj=hdf.select(metric)
        metric_data=metric_obj.get()
        
        for location in locations:
            [lat_range,long_range]=get_indices(location)
            temp=metric_data[lat_range[0]:lat_range[1]+1,long_range[0]:long_range[1]+1]
            temp=temp[np.where(temp>-3000)]
            if len(temp)<=0:
                data.append(["#".join([str(i) for i in location]),week,metric,-3000,-3000])
            else:
                data.append(["#".join([str(i) for i in location]),week,metric,round(np.mean(temp)),round(np.std(temp))]) 
    fp=open(OUTPUT_FILENAME,"a")
    for vector in data:
        fp.write(",".join([str(i) for i in vector])+"\n")
    fp.close()

1.2019.week1
1.2019.week2
2.2019.week1
2.2019.week2
3.2019.week1
3.2019.week2
4.2019.week1
4.2019.week2
5.2019.week1
5.2019.week2
6.2019.week1
6.2019.week2
7.2019.week2
8.2019.week1
8.2019.week2
9.2019.week1
9.2019.week2
10.2019.week1
10.2019.week2
11.2019.week1
11.2019.week2
12.2019.week1
12.2019.week2
1.2020.week1
1.2020.week2
2.2020.week1
2.2020.week2
3.2020.week1
3.2020.week2
4.2020.week1
4.2020.week2
5.2020.week1
5.2020.week2
6.2020.week1
6.2020.week2
7.2020.week1
8.2020.week1
8.2020.week2
9.2020.week1
9.2020.week2
10.2020.week1
10.2020.week2
11.2020.week1
11.2020.week2
12.2020.week1
12.2020.week2
1.2021.week1
1.2021.week2
2.2021.week1
2.2021.week2
3.2021.week1
3.2021.week2
4.2021.week1
4.2021.week2
5.2021.week1
5.2021.week2
6.2021.week1
6.2021.week2
7.2021.week2
8.2021.week1
8.2021.week2
9.2021.week1
9.2021.week2
10.2021.week1
